# 3.1. A Cheyenne Example

This example is meant to use one of the RDA datasets available on Cheyenne's GLADE storage system.

In [ ]:
import dask
import time

## How to Set Up Your Dask Distributed Cluster on Cheyenne

The easiest way to start a cluster on Cheyenne is using the `dask.distributed` `Client` object and a `PBSCluster` object from the `dask-jobqueue` package.

In [ ]:
from dask.distributed import Client
from dask_jobqueue import PBSCluster

cluster = PBSCluster(queue='regular', project='NIOW0001', interface='ib0',
                     processes=36, cores=36, memory='108GB', walltime='01:00:00')
client = Client(cluster)

### PBSCluster

The `PBSCluster` object defines what a "single worker" looks like on a PBS-based job queuing system, like the one on NCAR's Cheyenne.  Each "worker" is submitted to the PBS queuing system separately, as its own job.  So, the arguments of the `PBSCluster` object define what is requested for each "worker" via the PBS queuing system:

  - `queue`: The job queue to worker job is submitted to.
  - `project`: The project ID code for your allocation on Cheyenne.
  - `processes`: The number of separate Python processes associated with a single worker job.
  - `cores`: The number of cores given to a single worker.
  - `memory`: The memory available to a single worker.
  - `walltime`: The walltime to give to the worker job.
  
The setup above initializes a cluster with 0 nodes.  No jobs are submitted to the PBS queue, yet.  However, once we submit jobs to the PBS queue (with the `scale` command), each job will contain 9 processes (or workers) and 4 cores-per-process (for 36 cores total, or a full Cheyenne node) for 1 hour.

### Scaling up your cluster

Now that we have the cluster, and we have the SSH tunnel open to the dashboard, we can request some workers using the `scale` command of the cluster.

In [ ]:
cluster.scale(1)

In [ ]:
client

If you rerun the `client` command above, you will eventually notice the number of worker rise to 36 (and the number of cores rise to 144).  By submitted 4 PBS jobs to the queue, we eventually launched 36 Dask workers, with each worker having 4 cores.

### Dashboard

In the above output (of the `client` object), we see the number of workers and the total number of cores in the cluster.  Initially, we see 0 workers (and 0 cores, obviously).  Eventually, when your PBS jobs start, you should see 36 workers (144 cores).

We also see a pointer to the Dashboard.  Unfortunately, because we are accessing this cluster via an SSH tunnel, we cannot click this link and be taken to the Dashboard directly.  Instead, we have to note the Dashboard port number (probably 8787) and create *another* SSH tunnel to redirect that port to our browser.

In a termal, do that now.  It should look exactly like the other SSH tunnel command, but with the port numbers changed to match the port of the above link.  Then, open up another browser tab with *localhost:port*.

## Example: *Sea Surface Altimetry Data Analysis*

In [3]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
plt.rcParams['figure.figsize'] = (15,10)
%matplotlib inline

In [ ]:
lens_ocn_ssh_daily_rcp85 = '/glade/p_old/cesmLE/CESM-CAM5-BGC-LE/ocn/proc/tseries/daily/SSH_2/b.e11.BRCP85C5CNBDRD.f09_g16.*.pop.h.nday1.SSH_2.20060102-20801231.nc'

In [ ]:
ds = xr.open_mfdataset(lens_ocn_ssh_daily_rcp85, decode_times=False, chunks={'time':100})

In [ ]:
ds

### Examine Metadata
For those unfamiliar with this dataset, the variable metadata is very helpful for understanding what the variables actually represent

In [ ]:
for v in sorted(ds.data_vars):
    print('{:>20}: {}'.format(v, ds[v].attrs.get('long_name', '????????????????????????????????????')))

### Look a little deeper at the Sea Surface Height (SSH) Variable

In [ ]:
type(ds.SSH_2)

In [ ]:
type(ds.SSH_2.data)

In [ ]:
ds.SSH_2.data.dtype

In [ ]:
ds.SSH_2.data.numblocks

In [ ]:
ds.SSH_2.data.visualize()

### Visually Examine Some of the Data
Let's do a sanity check that the sea surface height (SSH) data looks reasonable:

In [ ]:
ds.SSH_2.sel(time=897.0, method='nearest').plot()

### How big is the dataset?

In [ ]:
print(ds.nbytes / 2**40, 'TB')

In [ ]:
print(ds.SSH_2.nbytes / 2**20, 'MB')

### Timeseries of Global Mean Sea Level
Here we make a simple yet fundamental calculation: the rate of increase of global mean sea level over the observational period.

In [ ]:
ssh_timeseries = ds.SSH_2.mean(dim=('nlat', 'nlon')).load()

In [ ]:
ssh_rolling_annual_mean = ssh_timeseries.rolling(time=365, center=True).mean()

In [ ]:
ssh_timeseries.plot(label='full data')
ssh_rolling_annual_mean.plot(label='rolling annual mean')
plt.legend()
plt.grid()

### Sea Level Variability
We can examine the natural variability in sea level by looking at its standard deviation in time.

In [ ]:
ssh_std = ds.SSH_2.std(dim='time').load()

In [ ]:
ssh_std.plot()

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree())
ssh_std.plot.contourf(ax=ax, transform=ccrs.PlateCarree())
ax.set_global()
ax.coastlines()